# Подготовка датасета

Модель YOLOv8 использует датасет в формата YOLOv5 PyTorch TXT.

001.txt
```console
1 0.617 0.3594420600858369 0.114 0.17381974248927037
1 0.094 0.38626609442060084 0.156 0.23605150214592274
1 0.295 0.3959227467811159 0.13 0.19527896995708155

class_id center_x center_y width height
```

![Пример формата YOLOv5](https://assets-global.website-files.com/5f6bc60e665f54545a1e52a5/614cd66c5d86816d057ef364_yolov5-coordinates.jpeg "Пример формата YOLOv5")


Так как аннотации сделаны в формате vgg json необходимо их трансформировать.
Для этого я переименовал в датаете директории на 1, 2, 3 ... в соответствии с названиями файлов аннотаций 1.json, 2.json ...


In [33]:
import json
import shutil
import cv2
import os
import random
import string

for i in range(1, 8):
    f = open(f'dataset/{i}.json')
    data = json.loads(f.read())
    for item in data.items():
        filename_rand = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        filename = '.'.join(item[1]['filename'].split('.')[:-1])
        file_ext = item[1]['filename'].split('.')[-1]
        img_path = f'dataset/images/{i}/{filename}.{file_ext}'
        if not os.path.isfile(img_path):
            continue
        image = cv2.imread(img_path)
        (image_height, image_width, channels) = image.shape
        lines = list()
        for region in item[1]['regions']:
            center_x = (region['shape_attributes']['x'] + region['shape_attributes']['width'] / 2) / image_width 
            center_y = (region['shape_attributes']['y'] + region['shape_attributes']['height'] / 2) / image_height
            region_width = region['shape_attributes']['width'] / image_width
            region_height = region['shape_attributes']['height'] / image_height
            lines.append(f'{i - 1} {center_x} {center_y} {region_width} {region_height}')
        if len(lines) == 0:
            continue
        f = open(f'dataset/train/labels/{filename_rand}.txt', 'w')
        f.write('\n'.join(lines))
        f.close()
        img_dst = f'dataset/train/images/{filename_rand}.{file_ext}'
        shutil.copyfile(img_path, img_dst)
        
            

# Обучение модели

Для обучения модели необходимо иметь видеокарту, при отсутствии можно воспользоваться гугл колабом
https://colab.research.google.com/drive/15-kkwkjGuQSWrxrHNLJZU8S8HtWEADGx?usp=sharing

# Использование модели

Запуск веб-сервера 

```console
python server.py
```

Отправлять POST-запрос с картинками в body (название параметра любое) на url http://localhost:8100/predict

Использование модели в питоне

In [ ]:
from ultralytics import YOLO

model = YOLO('best.pt')
# Можно на вход подать картинку или список картинок
results = model.predict('uploads/wiiojecpad-2ndary_efflorescence_09.jpg')
print(results)